In [80]:
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import norm, chi2
# import statsmodels.api as sm

# Import Data
df = pd.read_csv("../data/13222067_cleaned.csv")
df.head()

,jenis_kelamin,usia,pendidikan,pekerjaan,penghasilan,domisili,durasi_internet,aktivitas_online_meningkat,aktivitas_online_meningkat_3bln,layanan_aktif_1,...,bayar_online_5,bayar_online_6,bayar_online_7,bayar_online_8,keluhan_online_1,keluhan_online_2,keluhan_online_3,keluhan_online_4,keluhan_online_5,keluhan_online_6
0,Pria,47,SMA,Pekerja Serabutan,Rp 2 juta – Rp 5 juta,bandung,3.0,Sama saja,NaN,Mobile Banking,...,NaN,NaN,Melalui minimarket,NaN,Barang yang diperoleh tidak sesuai dengan spes...,Barang rusak/ salah tetapi tidak dapat dikemba...,NaN,NaN,NaN,NaN
1,Wanita,19,SMA,Pelajar / Mahasiswa,< Rp 2 juta,surabaya,10.0,Ya,Keperluan mengerjakan tugas,Mobile Banking,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pria,50,S1,Karyawan Swasta,Rp 5 juta – Rp 10 juta,jakarta,6.0,Ya,Melakukan video conference,Mobile Banking,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wanita,19,SMA,Pelajar / Mahasiswa,< Rp 2 juta,bandung,5.0,Ya,Mengikuti kelas online,Mobile Banking,...,NaN,NaN,NaN,NaN,Barang yang diperoleh tidak sesuai dengan spes...,Barang rusak/ salah tetapi tidak dapat dikemba...,NaN,NaN,NaN,NaN
4,Pria,28,S1,Karyawan Swasta,Rp 5 juta – Rp 10 juta,bandung,9.0,Sama saja,NaN,Mobile Banking,...,Transfer via ATM,NaN,NaN,NaN,NaN,NaN,Pembayaran sudah dilakukan; barang tidak tersedia,NaN,Pembayaran telah dilakukan tetapi tidak terdet...,NaN


In [81]:
# Confidence Interval
def countToSeries(df, column, normalize=True, threshold=0, otherLabel="lain"):
    value_count = df[column].value_counts(normalize=normalize)
    total = value_count.sum()
    newSeries = {}
    for idx, val in value_count.items():
        if(val/total < threshold/100):
            if(otherLabel in newSeries):
                newSeries[otherLabel] += val
            else:
                newSeries[otherLabel] = 0
        else:
            newSeries[idx] = val
    series1 = pd.Series(newSeries)
    return series1

def multipleChoiceToSeries(df, column_list, normalize=True):
    value_counts = {}
    total = df[column_list[0]].shape[0]
    for col in column_list:
        value_count = df[col].value_counts(dropna=True) 
        key = value_count.keys().tolist()[0]
        val = value_count.tolist()[0]
        if(normalize):
            value_counts[key] = val/total
        else:
            value_counts[key] = val
    value_counts_series = pd.Series(value_counts)
    return value_counts_series

    
def ci_p(phat_series, n, alpha=0.05):
    z_halfAlpha = norm.ppf(1-(alpha/2))
    lbDict = {}
    ubDict = {}
    for key, phat in phat_series.items():
        marginErr = z_halfAlpha*((phat*(1-phat)/n)**0.5)
        lbDict[key] = phat-marginErr
        ubDict[key] = phat+marginErr
    return (z_halfAlpha, pd.concat([phat_series.rename("Proportion"), pd.Series(lbDict, name="Lower"), pd.Series(ubDict, name="Upper")], axis=1))




# ecommerce_cols = ["ecommerce_1", "ecommerce_2", "ecommerce_3", "ecommerce_4", "ecommerce_5", "ecommerce_6"
#         , "ecommerce_7", "ecommerce_8", "ecommerce_9", "ecommerce_10", "ecommerce_11", "ecommerce_12"]
# seriesx = multipleChoiceToSeries(df, ecommerce_cols, normalize=True)
# print(seriesx)
# test = {
#     "Gopay":0.85,
#     "OVO":0.768,
#     "DANA":0.47,
#     "e-money":0.245,
#     "flazz":0.21
# }
# test = {"test": 0.4026}
# seriesx = pd.Series(test)
# print(ci_proportion(seriesx, 891, 0.05))
# ci_low, ci_upp = proportion_confint(value_counts_series, value_counts_series.sum(), alpha=0.05)
# print(ci_low)
# print(ci_upp)
# print("---")



In [85]:
def chiSqGoF(series, alpha=0.05): # Basically.. Is this series uniform???
    n = series.sum()
    Ei = 1/len(series)*n
    critRegion = 0
    for key, Oi in series.items():
        oiei2ei = ((Oi-Ei)**2 / Ei)
        critRegion += oiei2ei
    print(critRegion)

    chi_alpha = chi2.ppf(1-alpha, n-1)
    print(chi_alpha)

    # return Ei
        

df_test = pd.read_csv("../data/data_test_chisq.csv")
seriesTest = df_test["operator_pilihan"].value_counts()
print(seriesTest)
chiSqGoF(seriesTest, 0.05)


operator_pilihan
tel    5
ind    4
xl     3
Name: count, dtype: int64
0.5


NameError: name 'chi_halfalpha' is not defined